
Dataset description: This dataset contains information on the Battery Electric Vehicles (BEVs) and Plug-in Hybrid Electric Vehicles (PHEVs) that are currently registered with the Washington State Department of Licensing (DOL). This dataset was introduced as part of an official invitation-based competition on Kaggle. Our SVM model should answer the question "This is my car's model & make, along with a few other parameters, what price can this vehicle be brought or sold?”

Dataset link: https://drive.google.com/file/d/1kZ299dY3rKLvvnfTsaPtfrUbZb7k31of/view?usp=sharing

Import neccessary libraries

In [1]:
#To mute warnings
import warnings
warnings.filterwarnings("ignore")

In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.svm import SVR
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from pandas_profiling import ProfileReport
from sklearn.preprocessing import LabelEncoder
from scipy import stats

print(f' Libraries successfully imported')

 Libraries successfully imported


Import/Load Dataset

In [5]:
data = pd.read_csv("Electric_cars_dataset.csv", nrows = 10000)
data

,ID,VIN (1-10),County,City,State,ZIP Code,Model Year,Make,Model,Electric Vehicle Type,Clean Alternative Fuel Vehicle (CAFV) Eligibility,Electric Range,Base MSRP,Legislative District,DOL Vehicle ID,Vehicle Location,Electric Utility,Expected Price ($1k)
0,EV33174,5YJ3E1EC6L,Snohomish,LYNNWOOD,WA,98037.0,2020.0,TESLA,MODEL 3,Battery Electric Vehicle (BEV),Clean Alternative Fuel Vehicle Eligible,308,0,32.0,109821694,POINT (-122.287614 47.83874),PUGET SOUND ENERGY INC,50
1,EV40247,JN1AZ0CP8B,Skagit,BELLINGHAM,WA,98229.0,2011.0,NISSAN,LEAF,Battery Electric Vehicle (BEV),Clean Alternative Fuel Vehicle Eligible,73,0,40.0,137375528,POINT (-122.414936 48.709388),PUGET SOUND ENERGY INC,15
2,EV12248,WBY1Z2C56F,Pierce,TACOMA,WA,98422.0,2015.0,BMW,I3,Battery Electric Vehicle (BEV),Clean Alternative Fuel Vehicle Eligible,81,0,27.0,150627382,POINT (-122.396286 47.293138),BONNEVILLE POWER ADMINISTRATION||CITY OF TACOM...,18
3,EV55713,1G1RD6E44D,King,REDMOND,WA,98053.0,2013.0,CHEVROLET,VOLT,Plug-in Hybrid Electric Vehicle (PHEV),Clean Alternative Fuel Vehicle Eligible,38,0,45.0,258766301,POINT (-122.024951 47.670286),PUGET SOUND ENERGY INC||CITY OF TACOMA - (WA),33.9
4,EV28799,1G1FY6S05K,Pierce,PUYALLUP,WA,98375.0,2019.0,CHEVROLET,BOLT EV,Battery Electric Vehicle (BEV),Clean Alternative Fuel Vehicle Eligible,238,0,25.0,296998138,POINT (-122.321062 47.103797),BONNEVILLE POWER ADMINISTRATION||CITY OF TACOM...,41.78
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,EV65171,5YJ3E1EA1J,King,WOODINVILLE,WA,98072.0,2018.0,TESLA,MODEL 3,Battery Electric Vehicle (BEV),Clean Alternative Fuel Vehicle Eligible,215,0,45.0,127506480,POINT (-122.132064 47.759381),PUGET SOUND ENERGY INC||CITY OF TACOMA - (WA),69
9996,EV27827,1G1FY6S08M,Benton,KENNEWICK,WA,99337.0,2021.0,CHEVROLET,BOLT EV,Battery Electric Vehicle (BEV),Eligibility unknown as battery range has not b...,0,0,8.0,154783184,POINT (-119.102002 46.147282),BONNEVILLE POWER ADMINISTRATION||PUD NO 1 OF B...,36.5
9997,EV65846,5YJYGDED9M,Snohomish,BOTHELL,WA,98012.0,2021.0,TESLA,MODEL Y,Battery Electric Vehicle (BEV),Eligibility unknown as battery range has not b...,0,0,1.0,147162480,POINT (-122.201515 47.843376),PUGET SOUND ENERGY INC,73
9998,EV84119,1N4AZ0CP3D,King,REDMOND,WA,98052.0,2013.0,NISSAN,LEAF,Battery Electric Vehicle (BEV),Clean Alternative Fuel Vehicle Eligible,75,0,48.0,132149349,POINT (-122.122018 47.678465),PUGET SOUND ENERGY INC||CITY OF TACOMA - (WA),18


**Perform basic data exploration**

In [7]:
# Display first 5 rows

data.head()

,ID,VIN (1-10),County,City,State,ZIP Code,Model Year,Make,Model,Electric Vehicle Type,Clean Alternative Fuel Vehicle (CAFV) Eligibility,Electric Range,Base MSRP,Legislative District,DOL Vehicle ID,Vehicle Location,Electric Utility,Expected Price ($1k)
0,EV33174,5YJ3E1EC6L,Snohomish,LYNNWOOD,WA,98037.0,2020.0,TESLA,MODEL 3,Battery Electric Vehicle (BEV),Clean Alternative Fuel Vehicle Eligible,308,0,32.0,109821694,POINT (-122.287614 47.83874),PUGET SOUND ENERGY INC,50
1,EV40247,JN1AZ0CP8B,Skagit,BELLINGHAM,WA,98229.0,2011.0,NISSAN,LEAF,Battery Electric Vehicle (BEV),Clean Alternative Fuel Vehicle Eligible,73,0,40.0,137375528,POINT (-122.414936 48.709388),PUGET SOUND ENERGY INC,15
2,EV12248,WBY1Z2C56F,Pierce,TACOMA,WA,98422.0,2015.0,BMW,I3,Battery Electric Vehicle (BEV),Clean Alternative Fuel Vehicle Eligible,81,0,27.0,150627382,POINT (-122.396286 47.293138),BONNEVILLE POWER ADMINISTRATION||CITY OF TACOM...,18
3,EV55713,1G1RD6E44D,King,REDMOND,WA,98053.0,2013.0,CHEVROLET,VOLT,Plug-in Hybrid Electric Vehicle (PHEV),Clean Alternative Fuel Vehicle Eligible,38,0,45.0,258766301,POINT (-122.024951 47.670286),PUGET SOUND ENERGY INC||CITY OF TACOMA - (WA),33.9
4,EV28799,1G1FY6S05K,Pierce,PUYALLUP,WA,98375.0,2019.0,CHEVROLET,BOLT EV,Battery Electric Vehicle (BEV),Clean Alternative Fuel Vehicle Eligible,238,0,25.0,296998138,POINT (-122.321062 47.103797),BONNEVILLE POWER ADMINISTRATION||CITY OF TACOM...,41.78


In [9]:
#Display info about the dataset

data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 18 columns):
 #   Column                                             Non-Null Count  Dtype  
---  ------                                             --------------  -----  
 0   ID                                                 10000 non-null  object 
 1   VIN (1-10)                                         10000 non-null  object 
 2   County                                             9998 non-null   object 
 3   City                                               10000 non-null  object 
 4   State                                              9997 non-null   object 
 5   ZIP Code                                           10000 non-null  float64
 6   Model Year                                         10000 non-null  float64
 7   Make                                               10000 non-null  object 
 8   Model                                              9998 non-null   object 
 9   Electri

In [11]:
# perform descriptive analysis

data.describe().T

,count,mean,std,min,25%,50%,75%,max
ZIP Code,10000.0,9.816614e+04,2.594331e+03,6340.0,9.805200e+04,98121.0,9.837000e+04,99567.0
Model Year,10000.0,2.018199e+03,2.752838e+00,1993.0,2.017000e+03,2019.0,2.021000e+03,2022.0
Electric Range,10000.0,1.059118e+02,1.033843e+02,0.0,1.400000e+01,73.0,2.150000e+02,337.0
Base MSRP,10000.0,2.380468e+03,1.160957e+04,0.0,0.000000e+00,0.0,0.000000e+00,184400.0
Legislative District,9974.0,2.997644e+01,1.459728e+01,1.0,1.900000e+01,34.0,4.300000e+01,49.0
DOL Vehicle ID,10000.0,1.965264e+08,1.081728e+08,4777.0,1.361600e+08,173100472.5,2.264984e+08,478934571.0


In [13]:
data.shape

(10000, 18)

**PERFRORM PANDAS PROFILING**

In [15]:
profile = ProfileReport(data, title="Electric Vehicle Data Profile Report")
profile.to_widgets() 

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render widgets:   0%|          | 0/1 [00:00<?, ?it/s]

**CHECK FOR MISSING VALUES**

In [17]:
data.isnull().sum()

ID                                                     0
VIN (1-10)                                             0
County                                                 2
City                                                   0
State                                                  3
ZIP Code                                               0
Model Year                                             0
Make                                                   0
Model                                                  2
Electric Vehicle Type                                  0
Clean Alternative Fuel Vehicle (CAFV) Eligibility      0
Electric Range                                         0
Base MSRP                                              0
Legislative District                                  26
DOL Vehicle ID                                         0
Vehicle Location                                      84
Electric Utility                                     112
Expected Price ($1k)           

In [19]:
#Some reduntant columns will be dropped

data = data.drop(columns=['Legislative District', 'Vehicle Location', 'Electric Utility'])

In [21]:
data

,ID,VIN (1-10),County,City,State,ZIP Code,Model Year,Make,Model,Electric Vehicle Type,Clean Alternative Fuel Vehicle (CAFV) Eligibility,Electric Range,Base MSRP,DOL Vehicle ID,Expected Price ($1k)
0,EV33174,5YJ3E1EC6L,Snohomish,LYNNWOOD,WA,98037.0,2020.0,TESLA,MODEL 3,Battery Electric Vehicle (BEV),Clean Alternative Fuel Vehicle Eligible,308,0,109821694,50
1,EV40247,JN1AZ0CP8B,Skagit,BELLINGHAM,WA,98229.0,2011.0,NISSAN,LEAF,Battery Electric Vehicle (BEV),Clean Alternative Fuel Vehicle Eligible,73,0,137375528,15
2,EV12248,WBY1Z2C56F,Pierce,TACOMA,WA,98422.0,2015.0,BMW,I3,Battery Electric Vehicle (BEV),Clean Alternative Fuel Vehicle Eligible,81,0,150627382,18
3,EV55713,1G1RD6E44D,King,REDMOND,WA,98053.0,2013.0,CHEVROLET,VOLT,Plug-in Hybrid Electric Vehicle (PHEV),Clean Alternative Fuel Vehicle Eligible,38,0,258766301,33.9
4,EV28799,1G1FY6S05K,Pierce,PUYALLUP,WA,98375.0,2019.0,CHEVROLET,BOLT EV,Battery Electric Vehicle (BEV),Clean Alternative Fuel Vehicle Eligible,238,0,296998138,41.78
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,EV65171,5YJ3E1EA1J,King,WOODINVILLE,WA,98072.0,2018.0,TESLA,MODEL 3,Battery Electric Vehicle (BEV),Clean Alternative Fuel Vehicle Eligible,215,0,127506480,69
9996,EV27827,1G1FY6S08M,Benton,KENNEWICK,WA,99337.0,2021.0,CHEVROLET,BOLT EV,Battery Electric Vehicle (BEV),Eligibility unknown as battery range has not b...,0,0,154783184,36.5
9997,EV65846,5YJYGDED9M,Snohomish,BOTHELL,WA,98012.0,2021.0,TESLA,MODEL Y,Battery Electric Vehicle (BEV),Eligibility unknown as battery range has not b...,0,0,147162480,73
9998,EV84119,1N4AZ0CP3D,King,REDMOND,WA,98052.0,2013.0,NISSAN,LEAF,Battery Electric Vehicle (BEV),Clean Alternative Fuel Vehicle Eligible,75,0,132149349,18


In [23]:
#Fill missing values
data.fillna(data.mode().iloc[0], inplace=True)  # since we are dealing with catergorical data

In [25]:
# Removing Duplicates
data.drop_duplicates(inplace=True)
print("Shape of Dataset after removing duplicates:", data.shape) 

Shape of Dataset after removing duplicates: (10000, 15)


**HANDLE OUTLIERS**

In [27]:
# Handling Outliers
# Using Z-score method for outlier detection
z_scores = np.abs(stats.zscore(data.select_dtypes(include=[np.number])))
data = data[(z_scores < 3).all(axis=1)]  # Keep rows where all numeric columns are within 3 standard deviations
print("Shape after removing outliers:", data.shape)

Shape after removing outliers: (9702, 15)


**ENCODING CATERGORICAL VARIBLES**

In [101]:
#Check count

data['Expected Price ($1k)'].unique()

# Count of unique values
data['Expected Price ($1k)'].describe()

count    9700.000000
mean      100.291649
std        48.475188
min         0.000000
25%        53.000000
50%       111.000000
75%       148.000000
max       179.000000
Name: Expected Price ($1k), dtype: float64

In [85]:
# Assuming 'make' and 'model' are categorical features to be encoded
label_encoder = LabelEncoder()
data['Make'] = label_encoder.fit_transform(data['Make'])
data['Model'] = label_encoder.fit_transform(data['Model'])
data['Electric Range'] = label_encoder.fit_transform(data['Electric Range'])
data['Clean Alternative Fuel Vehicle (CAFV) Eligibility'] = label_encoder.fit_transform(data['Clean Alternative Fuel Vehicle (CAFV) Eligibility'])
data['Expected Price ($1k)'] = label_encoder.fit_transform(data['Expected Price ($1k)'])

**FEATURE ENGINEERING**

In [87]:
# Selecting Features and Target Variable, Assuming 'price' is the target variable
X = data[['Make', 'Model', 'Electric Range', 'Clean Alternative Fuel Vehicle (CAFV) Eligibility']]  # Features
y = data['Expected Price ($1k)']               # Target variable

In [89]:
# Splitting Data into Training and Testing Sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [91]:
# Check unique values in the target column to identify non-numeric values
print(y.unique())

[124  20  33  91 111  26  82 143  39  66 107 106 110  63 153 158 152 105
  24  88 142 148  25  77  41  17  93 170 131 156  46 162 119 134 149  40
  19  60  53  99 154  70 139  65  62 138  31   7  83  98 178  51  14  78
  36  54  23  69  34  81 113 179 114   4 141 102 112 140 137   9 159  13
 151 104  27  75  48  35 167 108 166  52 160  43  44  72  92  85  97 118
  79 168  59 121  55 101 146 100 126  64 173   0  37 103 144  50   2 177
  47   3  61  22  30  38  80 147 169 123 127  90  11 176  49  57  71 136
 174 122  28  21   1  94  16 129  84  56 150 175 133 157  10 120  74 128
 163  15   8  67  12  58  45  18   6 116  42  76 115  86 161  73 171 135
 165 117  32 172 109 155 125   5  68  96 130 145  95  29  89  87 164 132]


In [93]:
# Building and Training the SVM Model
svm_model = SVR(kernel='rbf')  
svm_model.fit(X_train, y_train)

SVR()

In [95]:
# Model Evaluation
y_pred = svm_model.predict(X_test)
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

In [97]:
print("\nModel Performance:")
print("Mean Absolute Error (MAE):", mae)
print("Mean Squared Error (MSE):", mse)
print("R-squared (R²):", r2)


Model Performance:
Mean Absolute Error (MAE): 19.226820218049866
Mean Squared Error (MSE): 802.23220442977
R-squared (R²): 0.661697274116708
